In [1]:
import numpy as np
import math

In [2]:
v_states = [
            [1,1,1,0,0,0,0,0,0],
            [0,0,0,1,1,1,0,0,0],
            [0,0,0,0,0,0,1,1,1],
            [1,0,0,1,0,0,1,0,0],
            [0,1,0,0,1,0,0,1,0],
            [0,0,1,0,0,1,0,0,1],
            [1,0,0,0,1,0,0,0,1],
            [0,0,1,0,1,0,1,0,0],
            ]

def check_victory(s):
    # s is a length 9 array
    x = s == 1
    o = s == 2
    sx = v_states @ x
    so = v_states @ o
    if sx.max() == 3:
        return 1
    elif so.max() == 3:
        return -1
    else:
        return 0

def check_victory_mat(s):
    # s is matrix of n x 9 dimension
    # matrix product with vicory states: s @ v = n x 9 @ 9 x 8 -> n x 8 
    # a value of 3 in the output means a victory 
    x = s == 1
    o = s == 2
    sx = v_states @ np.transpose(x)
    so = v_states @ np.transpose(o)
    sx = sx.max(axis = 0)
    so = so.max(axis = 0)
    sx = (sx == 3) * 1
    so = (so == 3) * -1
    return so + sx
   
s_test = np.array([[2,1,1,0,0,1,2,2,1],
                  [2,1,2,0,0,1,2,2,1],
                  [2,1,2,0,0,2,2,2,2]])
for s in s_test:
    print(check_victory(s))

check_victory_mat(s_test)    

1
0
-1


array([ 1,  0, -1])

In [3]:
def print_board(s):
    x = ['-', 'x', 'o']
    print(f' [{x[s[0]]} {x[s[1]]} {x[s[2]]}] \n [{x[s[3]]} {x[s[4]]} {x[s[5]]}] \n [{x[s[6]]} {x[s[7]]} {x[s[8]]}] \n')

def print_states(states):
    print(len(states)) 
    for s in states:
        print(len(s)) 

# get all tick tack toe states. 0 is empty, 1 is x and 2 is o
board = [0 for i in range(9)]
states = [[board]]
current_round_states = [board]
next_round_states = []
connections = []
current_connections = []
# iteration over all steps to fill the board
for i in range(9):
    #iteration over all states of the previous step
    for j, s in enumerate(current_round_states):
        # check if the step is even. If yes, add a x otherwise an o
        if i % 2 == 0:
            x = 1
        else:
            x = 2    
        # go through all fields
        c = []
        # check if state was victorios. Only calculate next states if the state wasn't victorios 
        if check_victory(np.array(s)) == 0:
            for k in range(9):
                new_state = list(s).copy()
                # check if field is taken
                if s[k] == 0:
                    new_state[k] = x
                    if new_state in next_round_states:
                        c.append(next_round_states.index(new_state))
                    else:
                        next_round_states.append(new_state)
                        c.append(next_round_states.index(new_state))
        current_connections.append(c)
    connections.append(current_connections)
    current_connections = []
    states.append(next_round_states)
    # remove states that won from the next round
    victories = check_victory_mat(np.array(next_round_states))
    current_round_states = np.array(next_round_states)
    next_round_states = []

    
print_states(states)
print('\n')
print_states(connections)

10
1
9
72
252
756
1260
1520
1140
390
78


9
1
9
72
252
756
1260
1520
1140
390


In [4]:
V = []
for s in states[0:]:
    V.append(np.random.rand(len(s)))

print_states(V)


10
1
9
72
252
756
1260
1520
1140
390
78


In [5]:
gamma = 0.5
def calc_V(V, turn, i, verbose = False):

    actions = connections[turn][i] 
    V_new = []
    for a in actions:
        s = states[turn + 1][a]
        reward = check_victory(np.array(s))
        if reward == 0 and turn != 8:
            V_new.append( gamma * V[turn + 1][a])

        else:
            V_new.append(reward)
    
    if verbose == True:
        print(V_new)

    if turn%2==0:    
        V_max = max(V_new)
        a = np.argmax(V_new)
    else: 
        V_max = min(V_new)
        a = np.argmin(V_new)
        
    return V_max, actions[a]

In [6]:
turn = 4
state_index = 49
V_new, a_best = calc_V(V, turn,state_index,verbose=True)
print(V[turn][state_index])
print_board(states[turn][state_index])
print(V_new, a_best)
print_board(states[turn + 1][a_best])
for a in connections[turn][state_index]:
    print(a)
    print_board(states[turn + 1][a])

[0.11417974102206002, 0.38228900258383186, 1, 0.18792591753391458, 0.13089380676566087]
0.2150349557605865
 [x - o] 
 [x - o] 
 [- - -] 

1 140
 [x - o] 
 [x - o] 
 [x - -] 

115
 [x x o] 
 [x - o] 
 [- - -] 

139
 [x - o] 
 [x x o] 
 [- - -] 

140
 [x - o] 
 [x - o] 
 [x - -] 

141
 [x - o] 
 [x - o] 
 [- x -] 

142
 [x - o] 
 [x - o] 
 [- - x] 



In [7]:
def get_best_action(V, turn, state_index):
    actions = connections[turn][state_index]
    v_arr = [V[turn + 1][a] for a in actions]
    best_action = actions[np.argmax(v_arr)] 
    return best_action
  


def play_game(V, verbose = False):
    a = 0
    for t in range(9):
        s = states[t][a]
        reward = check_victory(np.array(s))
        if verbose:
            print(f'{t=}, {a=}')
            print_board(s)
        if reward != 0:
            return reward    
        if t%2 != 0:
            next_moves = connections[t][a]
            a = np.random.choice(next_moves)
        if t%2 == 0:
            #a = get_best_action(V, t, a)  
            V_new, a = calc_V(V, t,a)

    return 0


n = 1000
rewards = sum([play_game(V) for i in range(n)])/n
print(rewards)

0.687


In [15]:
gamma = 0.5 # discount for future rewards
epsilon = 1.0 # epsilon determines the exploration of not optimal actions. Higher epsilon = more exploration. between 0 and 1

def choose_next_action(best_action, possible_actions):
    if np.random.random_sample() < epsilon:
        action = np.random.choice(possible_actions)
    else: 
        action = best_action
    return action

def assign_V(V, V_new, turn, state_index):
    residuum = np.abs(V[turn][state_index] - V_new)
    V[turn][state_index] = V_new
    return residuum

def initialize_V():
    V = []
    for s in states[0:]:
        V.append(np.zeros(len(s)))  
    return V      

def iterate_V(V):
    state_index = 0 # innitial starting board
    for turn in np.arange(0,9):
        possible_actions = connections[turn][state_index]
        V_new, best_action = calc_V(V, turn, state_index)
        V[turn][state_index] = V_new
        if turn%2 == 0:
            action = choose_next_action(best_action, possible_actions)    
        else: 
            action = np.random.choice(possible_actions)
        state_index = action
        if check_victory(np.array(states[turn + 1][action])) != 0:
            break  
    return V

V = initialize_V()
for i in range(10000):
    residuum = []
    V = iterate_V(V)
    if i > 0 and i%100 == 0:
        n = 1000
        rewards = sum([play_game(V) for i in range(n)])/n
        print(rewards)

0.734
0.733
0.724
0.737
0.738
0.728
0.734
0.762
0.782
0.698
0.785
0.839
0.812
0.775
0.787
0.768
0.815
0.84
0.8
0.827
0.785
0.803
0.804
0.798
0.811
0.799
0.882
0.892
0.848
0.883
0.885
0.886
0.872
0.92
0.94
0.975
0.979
0.985
0.972
0.985
0.987
0.991
0.991
0.989
0.981
0.995
0.996
0.994
0.991
0.994
0.995
0.993
0.992
0.993
0.994
0.996
0.993
0.996
0.995
0.995
0.993
0.996
0.999
0.996
0.994
0.996
0.996
0.994
0.995
0.997
0.991
0.996
0.997
0.993
0.996
0.996
0.991
0.996
0.993
0.994
0.996
0.996
0.989
0.997
0.997
0.996
0.993
0.994
0.991
0.993
0.994
0.995
0.997
0.996
0.995
0.995
0.995
0.995
0.995


In [16]:
play_game(V, verbose=True)

t=0, a=0
 [- - -] 
 [- - -] 
 [- - -] 

t=1, a=0
 [x - -] 
 [- - -] 
 [- - -] 

t=2, a=0
 [x o -] 
 [- - -] 
 [- - -] 

t=3, a=1
 [x o -] 
 [x - -] 
 [- - -] 

t=4, a=10
 [x o -] 
 [x - -] 
 [- o -] 

t=5, a=49
 [x o -] 
 [x - -] 
 [x o -] 



1

In [17]:
import pickle

save_dict = {'states':states,
             'connections': connections,
             'valuetable': V}

with open('TTT_model.pkl', 'wb') as f:  # open a text file
    pickle.dump(save_dict, f) # serialize the list
f.close()


In [18]:
V

[array([0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625, 0.0625, 0.0625,
        0.    , 0.    , 0.0625, 0.    , 0.0625, 0.0625, 0.    , 0.0625,
        0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625, 0.0625, 0.0625,
        0.    , 0.0625, 0.0625, 0.    , 0.    , 0.    , 0.0625, 0.0625,
        0.    , 0.0625, 0.    , 0.0625, 0.0625, 0.    , 0.0625, 0.    ,
        0.0625, 0.0625, 0.    , 0.    , 0.    , 0.0625, 0.0625, 0.    ,
        0.0625, 0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625, 0.0625,
        0.0625, 0.    , 0.0625, 0.0625, 0.    , 0.0625, 0.    , 0.    ,
        0.0625, 0.0625, 0.0625, 0.0625, 0.    , 0.0625, 0.0625, 0.0625]),
 array([ 0.    ,  0.125 ,  0.125 ,  0.    ,  0.125 ,  0.    ,  0.    ,
        -0.0625,  0.125 ,  0.    ,  0.    ,  0.125 ,  0.    ,  0.125 ,
         0.125 ,  0.125 ,  0.125 ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
      